In [ ]:
import warnings #忽略警告
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf # 若沒安裝tensorflow，請複製右串：pip install tensorflow
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout,BatchNormalization,TimeDistributed,Flatten,Conv1D,InputLayer
from tensorflow.keras import optimizers
import os
from sklearn.preprocessing import MinMaxScaler # 若沒安裝sklearn，請複製右串：pip install -U scikit-learn / conda install -c conda-forge scikit-learn
from tcn import TCN # 若沒安裝TCN，請複製右串：pip install keras-tcn

#以下程式碼功能為開啟GPU，開啟前須確認自己電腦有沒有顯卡，另外需在環境裡安裝cudnn、cudatoolkit、tensorflow-gpu
#步驟如下：
#(1) 在視窗搜尋Anaconda Prompt (anaconda3)
#(2) 複製右串：- conda install cudnn=7.4.2
#(3) 複製右串：- conda install cudatoolkit=10.0.130
#(4) 複製右串：- conda install tensorflow-gpu=1.13.1

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) # 若裝置有顯示GPU，代表可以使用GPU了
tf.test.is_gpu_available()
tf.config.list_physical_devices('GPU')
warnings.filterwarnings('ignore')
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.8
session = tf.compat.v1.Session(config=config)

# 參數設定

In [ ]:
n_in = 72*6 #以前72小時去預測PM10
n_out = 12*6 #預測未來12小時PM10
transfor_to_hour = 12
n_vars = 4 #特徵數量
tw = 1*6 #時間窗格(shift)
cut_time = 5 # 6-1
n_neurons = 64 #神經元數量
n_batch = 32
n_epoch = 100
Dp = 0.2 #Dropout
learning_rate = 0.001
train_proportion = 0.8 #以0.8作為切分訓練集和測試集的依據
ST = '麥寮' #空品監測站：麥寮、臺西、崙背，河川揚塵測站：旭光、義賢
activation = 'tanh'
loss = 'mae' #訓練模型的衡量指標，用來回饋模型修正參數
metrics = ['mse'] #訓練模型的評價函數，評估結果不會用來訓練模型
opt = tf.keras.optimizers.Adam() #優化器
path = os.getcwd()

In [ ]:
data = pd.read_csv('D:/'+ST+'(已清洗).csv')

# 針對每一小時進行線性插值法

In [ ]:
def ten_mins_pm10(data,start_time,end_time):
    pm10_tenmin=[]
    wind_speed_tenmin=[]
    wind_direc_tenmin=[]
    amb_temp_tenmin=[]
    rh_tenmin=[]
    time = pd.date_range(start=start_time,end=end_time,freq='10min')[:-7]
    pm10_2020_np = data.iloc[:,1:].values
    for i in range(len(pm10_2020_np)-1):
        pm10_ten_min=int((pm10_2020_np[i+1,0]-pm10_2020_np[i,0])/6)
        wind_speed_ten_min=int((pm10_2020_np[i+1,1]-pm10_2020_np[i,1])/6)
        wind_direc_ten_min=int((pm10_2020_np[i+1,2]-pm10_2020_np[i,2])/6)
        amb_temp_ten_min=int((pm10_2020_np[i+1,3]-pm10_2020_np[i,3])/6)
        rh_tenmin_ten_min=int((pm10_2020_np[i+1,4]-pm10_2020_np[i,4])/6)
        for j in range(6):
            pm10_tenmin.append(pm10_2020_np[i,0]+pm10_ten_min*j)
            wind_speed_tenmin.append(pm10_2020_np[i,1]+wind_speed_ten_min*j)
            wind_direc_tenmin.append(pm10_2020_np[i,2]+wind_direc_ten_min*j)
            amb_temp_tenmin.append(pm10_2020_np[i,3]+amb_temp_ten_min*j)
            rh_tenmin.append(pm10_2020_np[i,4]+rh_tenmin_ten_min*j)
    data = {'Time':list(time),'pm10':pm10_tenmin,'wind_speed':wind_speed_tenmin,'wind_direc':wind_direc_tenmin,'amb_temp':amb_temp_tenmin,'rh':rh_tenmin}
    data = pd.DataFrame(data=data)
    return data

In [ ]:
data = ten_mins_pm10(data,'1/1/2020','1/1/2021')
data

# 所有特徵資料的分布情形

In [ ]:
import seaborn as sns

data_col = data.columns[1:]
for i in range(len(data_col)):
    sns.distplot(x=list(data[data_col[i]]),color='green')
    plt.title(data_col[i],fontsize=24)
    plt.show()
    plt.close()

# 天氣因子是否要用預報資料 

In [ ]:
# 若沒有要使用預報資料，請忽略此行，直接跳到 "選擇輸入為多/單特徵" 繼續執行

next_time = 12*6 # 未來多長時間的天氣因子當輸入
weather_data = data.iloc[next_time:,2:] #
weather_data.reset_index(inplace=True)
weather_data = weather_data.drop(['index'],axis=1)
pm10_data = data[:-next_time]
pm10_data.iloc[:,2:] = weather_data.values
data = pm10_data
data

# 選擇輸入為多/單特徵

In [ ]:
def choose_feature(data,one_feature=False,mutiple_feature=False,drop_feature=False,want_to_drop_feature=None): # one_feature 和 mutiple_feature只能有一個為True
    if mutiple_feature == True:
        if drop_feature == True: # 未必每個特徵都會用到，若drop_feature=True，就必須填入要捨棄掉的特徵
            data = data.drop(want_to_drop_feature,axis=1)
        feature_data = data.iloc[:,1:]
        time = pd.DataFrame(list(data['Time']),columns=['Time'])
        output_data = pd.DataFrame(list(data['pm10']),columns=['pm10'])
    if one_feature == True:
        feature_data = pd.DataFrame(data.loc[:,'pm10'])
        time = pd.DataFrame(list(data['Time']),columns=['Time'])
        output_data = feature_data
    return feature_data,time,output_data # 產出：feature_data(模型input)、time、output_data(模型output)

In [ ]:
prepared_data = choose_feature(data,mutiple_feature=True,drop_feature=True,want_to_drop_feature=['wind_direc','amb_temp','rh'])#,drop_feature=True,want_to_drop_feature=['wind_direc'])

# 資料正規化/Log

In [ ]:
def model_prepare(intput,output=None,not_process=False,maxmin_process=False,log_process=False):
    if not_process == True: # 資料不進行處理
        intput = intput.values
        output = output.values
        scaler1 = None
    if maxmin_process == True: # 正規化
        intput = intput.values
        intput = intput.astype('float64')
        scaler = MinMaxScaler(feature_range=(0, 1))
        intput = scaler.fit_transform(intput)
        output = output.values
        output = output.astype('float64')
        scaler1 = MinMaxScaler(feature_range=(0, 1))
        output = scaler1.fit_transform(output)
    if log_process == True: # 當log_process=True時，output必須為None
        data_col = intput.columns
        for i in range(len(data_col)):
            for j in range(len(intput)): #資料轉log
                if intput.loc[j,data_col[i]] != 0:
                    intput.loc[j,data_col[i]] = np.log(intput.loc[j,data_col[i]])
        output = pd.DataFrame(intput.loc[:,'pm10']).values
        intput = intput.values
        scaler1 = None
    return intput,output,scaler1 # 產出：intput,output,scaler1(MinMaxScaler正規化壓縮器)

In [ ]:
prepare_data = model_prepare(prepared_data[0],prepared_data[2],not_process=True)

# 時間切割

In [ ]:
X=[]
Y=[]
T=[]

for i in range(n_in,len(prepare_data[1]),tw): #模型輸出(Y)的時間切割
    if len(prepare_data[1][i:i+n_out-cut_time]) == n_out-cut_time:
        Y.append(prepare_data[1][i:i+n_out-cut_time])
        T.append(choose_feature(data,mutiple_feature=True)[1][i:i+n_out-cut_time])
for i in range(0,len(prepare_data[0]),tw): #模型輸入(X)的時間切割
    if len(prepare_data[0][i:i+n_in-cut_time]) == n_in-cut_time:
        X.append(prepare_data[0][i:i+n_in-cut_time])
X = np.array(X)
X = X[:len(Y)] #由於在時間切割上有額外設置時間窗格進行移動，會導致X和Y資料數量不一致，所以以Y數量為主
Y = np.array(Y)
n_train = round(X.shape[0]*train_proportion)
Xtest_time = T[n_train:]
X_test_time = np.array(Xtest_time)
X_test_time = np.reshape(X_test_time,(X_test_time.shape[0],X_test_time.shape[1]))
train_X,test_X = X[:n_train],X[n_train:]
train_y,test_y = Y[:n_train],Y[n_train:]

In [ ]:
train_XX = train_X
test_XX = test_X
train_yy = train_y
test_yy = test_y

# model_1：LSTM+TimeDistributed

In [ ]:
model_1 = Sequential()
model_1.add(LSTM(units = n_neurons,activation=activation,return_sequences=True,input_shape=(train_XX.shape[1], train_XX.shape[2])))
model_1.add(Dropout(Dp))
model_1.add(LSTM(units = n_neurons,activation=activation,return_sequences=True))
model_1.add(BatchNormalization())
model_1.add(Dense(train_yy.shape[-2]))
model_1.add(TimeDistributed(Dense(1, activation='relu')))
model_1.add(Flatten())
model_1.add(Dense(train_yy.shape[-2]))
model_1.compile(loss=loss,optimizer=opt,metrics=metrics)
model_1.summary()
history = model_1.fit(train_XX, train_yy, epochs=n_epoch, batch_size=n_batch, verbose=1,shuffle=False)

In [ ]:
def model_inverse(train_XX,train_yy,test_XX,test_yy,model,scaler1=None,not_process=False,maxmin_process=False,log_process=False):
    pred_train_data = model.predict(train_XX)
    pred_test_data = model.predict(test_XX)
    trainy = []
    testy = []
    for i in range(len(train_yy)):
        trainy.append(train_yy[i].flatten())
    for i in range(len(test_yy)):
        testy.append(test_yy[i].flatten()) 
    train_yy = trainy
    train_yy = np.array(train_yy)
    test_yy = testy
    test_yy = np.array(test_yy)
    if not_process == True:
        pass
    if maxmin_process == True: # 當maxmin_scaler=True時，必須輸入scaler1
        scale_new = MinMaxScaler()
        scale_new.min_, scale_new.scale_ = scaler1.min_[0], scaler1.scale_[0]
        pred_train_data = scale_new.inverse_transform(pred_train_data)
        pred_test_data = scale_new.inverse_transform(pred_test_data)
        train_yy = scale_new.inverse_transform(train_yy)
        test_yy = scale_new.inverse_transform(test_yy)
    if log_process == True:
        for i in range(train_yy.shape[0]):
            for j in range(train_yy.shape[1]):
                if train_yy[i][j] != 0:
                    train_yy[i][j] = np.exp(train_yy[i][j])
                if pred_train_data[i][j] != 0:
                    pred_train_data[i][j] = np.exp(pred_train_data[i][j])
        for i in range(test_yy.shape[0]):
            for j in range(test_yy.shape[1]):
                if test_yy[i][j] != 0:
                    test_yy[i][j] = np.exp(test_yy[i][j])
                if pred_test_data[i][j] != 0:
                    pred_test_data[i][j] = np.exp(pred_test_data[i][j])
    return train_yy,test_yy,pred_train_data,pred_test_data

In [ ]:
image = model_inverse(train_XX,train_yy,test_XX,test_yy,model_1,prepare_data[2],maxmin_process=True) # 改模型參數

In [ ]:
# 10 min
number = np.random.randint(image[1].shape[-1])
plt.figure(figsize=(5,5))
plt.plot(image[1].T[number])
plt.plot(image[3].T[number])
plt.title('Testing',fontproperties="SimSun",fontsize=24)
# plt.savefig('D:\績效圖/'+ST+'_預測未來'+str(int(n_out/6))+'小時(Testing)(not_process)(TCN).png')
plt.show()
plt.close()

In [ ]:
pm10_10_act = image[1]
pm10_10_pred = image[3]

rmse = []
mape = []

for i in range(pm10_10_act.shape[0]):
    for j in range(pm10_10_act.shape[1]):
        rmse.append((pm10_10_pred[i][j]-pm10_10_act[i][j])**2)
        if pm10_10_act[i][j] == 0:
            mape.append(abs(pm10_10_pred[i][j]-pm10_10_act[i][j]))
        else:
            mape.append(abs((pm10_10_pred[i][j]-pm10_10_act[i][j])/pm10_10_act[i][j]))
            
rmse = np.sqrt(np.mean(rmse))
mape = np.mean(mape)*100

print('RMSE:'+str(rmse))
print('MAPE:'+str(mape))

In [ ]:
data1 = pd.read_csv('D:/'+ST+'(已清洗).csv')
time = pd.date_range(start='1/1/2020',end='1/1/2021',freq='H')[:-1]
data1['Time'] = time

num = []
for i in range(len(X_test_time[0])):
    for j in range(len(data1)):
        if data1.loc[j,'Time'] == X_test_time[0][i]:
            num.append(j)
            break
num = num[0]

data2 = data1[num:]
data2.reset_index(inplace=True)
data2 = data2.drop(['index'],axis=1)
data2_time = list(data2['Time'])

a1 = []
b1 = []
c1 = []
for i in range(image[1].shape[0]):
    for j in range(0,image[1].shape[1],tw):
        if j == 0:
            a1.append(X_test_time[i][j])
            b1.append(image[1][i][j])
            c1.append(image[3][i][j])
        else:
            a1.append(X_test_time[i][j])
            b1.append(np.mean(image[1][i][j-tw:j]))
            c1.append(np.mean(image[3][i][j-tw:j]))

act1 = np.array(b1)
pred1 = np.array(c1)
transfor_act1 = np.reshape(act1,(int(len(act1)/transfor_to_hour),transfor_to_hour))
transfor_pred1 = np.reshape(pred1,(int(len(pred1)/transfor_to_hour),transfor_to_hour))

In [ ]:
# 平均1小時
number = np.random.randint(transfor_act1.shape[-1])
plt.figure(figsize=(5,5))
plt.plot(transfor_act1.T[number])
plt.plot(transfor_pred1.T[number])
plt.title('Testing',fontproperties="SimSun",fontsize=24)
# plt.savefig('D:\績效圖/'+ST+'_預測未來'+str(int(n_out/6))+'小時(Testing)(not_process)(TCN).png')
plt.show()
plt.close()

In [ ]:
# RMSE公式：np.sqrt(np.mean((預測值-實際值)^2))
# MAPE公式：100*(np.mean(abs((預測值-實際值)/實際值)))

rmse = []
mape = []

for i in range(transfor_act1.shape[0]-1):
    for j in range(transfor_act1.shape[1]):
        rmse.append((transfor_pred1[i][j]-transfor_act1[i][j])**2)
        if transfor_act1[i][j] == 0:
            mape.append(abs(transfor_pred1[i][j]-transfor_act1[i][j]))
        else:
            mape.append(abs((transfor_pred1[i][j]-transfor_act1[i][j])/transfor_act1[i][j]))
            
rmse = np.sqrt(np.mean(rmse))
mape = np.mean(mape)*100

print('RMSE:'+str(rmse))
print('MAPE:'+str(mape))

In [ ]:
a = []
b = []
c = []
for i in range(image[1].shape[0]):
    for j in range(0,image[1].shape[1],tw):
        a.append(X_test_time[i][j])
        b.append(image[1][i][j])
        c.append(image[3][i][j])

act = np.array(b)
pred = np.array(c)
transfor_act = np.reshape(act,(int(len(act)/transfor_to_hour),transfor_to_hour))
transfor_pred = np.reshape(pred,(int(len(pred)/transfor_to_hour),transfor_to_hour))

In [ ]:
def make_plot(train_yy,test_yy,pred_train_data,pred_test_data):
    number = np.random.randint(test_yy.shape[-1])
    plt.figure(figsize=(5,5))
    plt.plot(train_yy.T[number])
    plt.plot(pred_train_data.T[number])
    plt.title('Training',fontproperties="SimSun",fontsize=24)
    plt.savefig('D:\績效圖/'+ST+'_預測未來'+str(int(n_out/6))+'小時(Training)(maxmin_process)(LSTM).png')
    plt.show()
    plt.close()
    plt.figure(figsize=(5,5))
    plt.plot(test_yy.T[number])
    plt.plot(pred_test_data.T[number])
    plt.title('Testing',fontproperties="SimSun",fontsize=24)
    plt.savefig('D:\績效圖/'+ST+'_預測未來'+str(int(n_out/6))+'小時(Testing)(maxmin_process)(LSTM).png')
    plt.show()
    plt.close()

In [ ]:
make_plot(image[0],transfor_act,image[2],transfor_pred)

In [ ]:
# RMSE公式：np.sqrt(np.mean((預測值-實際值)^2))
# MAPE公式：100*(np.mean(abs((預測值-實際值)/實際值)))

rmse = []
mape = []

for i in range(transfor_act.shape[0]):
    for j in range(transfor_act.shape[1]):
        rmse.append((transfor_pred[i][j]-transfor_act[i][j])**2)
        if transfor_act[i][j] == 0:
            mape.append(abs(transfor_pred[i][j]-transfor_act[i][j]))
        else:
            mape.append(abs((transfor_pred[i][j]-transfor_act[i][j])/transfor_act[i][j]))
            
rmse = np.sqrt(np.mean(rmse))
mape = np.mean(mape)*100

print('RMSE:'+str(rmse))
print('MAPE:'+str(mape))

# model_2：Conv1D

In [ ]:
train_XX = train_X
test_XX = test_X
train_yy = train_y
test_yy = test_y

In [ ]:
model_2=Sequential()
model_2.add(InputLayer(input_shape=(train_XX.shape[1], train_XX.shape[2])))
for rate in (1,2,4,8)*2:
    model_2.add(Conv1D(filters=24, kernel_size=2, padding='causal', activation='relu', dilation_rate=rate))
model_2.add(Conv1D(filters=10, kernel_size=1))
model_2.add(Flatten())
model_2.add(Dense(train_yy.shape[-2]))
model_2.compile(loss=loss,optimizer=opt,metrics=metrics)
model_2.summary()
history = model_2.fit(train_XX, train_yy, epochs=n_epoch, batch_size=n_batch, verbose=1,shuffle=False)

In [ ]:
image = model_inverse(train_XX,train_yy,test_XX,test_yy,model_2,prepare_data[2],maxmin_process=True) # 改模型參數

In [ ]:
# 10 min
number = np.random.randint(image[1].shape[-1])
plt.figure(figsize=(5,5))
plt.plot(image[1].T[number])
plt.plot(image[3].T[number])
plt.title('Testing',fontproperties="SimSun",fontsize=24)
# plt.savefig('D:\績效圖/'+ST+'_預測未來'+str(int(n_out/6))+'小時(Testing)(not_process)(TCN).png')
plt.show()
plt.close()

In [ ]:
pm10_10_act = image[1]
pm10_10_pred = image[3]

rmse = []
mape = []

for i in range(pm10_10_act.shape[0]):
    for j in range(pm10_10_act.shape[1]):
        rmse.append((pm10_10_pred[i][j]-pm10_10_act[i][j])**2)
        if pm10_10_act[i][j] == 0:
            mape.append(abs(pm10_10_pred[i][j]-pm10_10_act[i][j]))
        else:
            mape.append(abs((pm10_10_pred[i][j]-pm10_10_act[i][j])/pm10_10_act[i][j]))
            
rmse = np.sqrt(np.mean(rmse))
mape = np.mean(mape)*100

print('RMSE:'+str(rmse))
print('MAPE:'+str(mape))

In [ ]:
data1 = pd.read_csv('D:/'+ST+'(已清洗).csv')
time = pd.date_range(start='1/1/2020',end='1/1/2021',freq='H')[:-1]
data1['Time'] = time

num = []
for i in range(len(X_test_time[0])):
    for j in range(len(data1)):
        if data1.loc[j,'Time'] == X_test_time[0][i]:
            num.append(j)
            break
num = num[0]

data2 = data1[num:]
data2.reset_index(inplace=True)
data2 = data2.drop(['index'],axis=1)
data2_time = list(data2['Time'])

a1 = []
b1 = []
c1 = []
for i in range(image[1].shape[0]):
    for j in range(0,image[1].shape[1],tw):
        if j == 0:
            a1.append(X_test_time[i][j])
            b1.append(image[1][i][j])
            c1.append(image[3][i][j])
        else:
            a1.append(X_test_time[i][j])
            b1.append(np.mean(image[1][i][j-tw:j]))
            c1.append(np.mean(image[3][i][j-tw:j]))

act1 = np.array(b1)
pred1 = np.array(c1)
transfor_act1 = np.reshape(act1,(int(len(act1)/transfor_to_hour),transfor_to_hour))
transfor_pred1 = np.reshape(pred1,(int(len(pred1)/transfor_to_hour),transfor_to_hour))

In [ ]:
# 平均1小時
number = np.random.randint(transfor_act1.shape[-1])
plt.figure(figsize=(5,5))
plt.plot(transfor_act1.T[number])
plt.plot(transfor_pred1.T[number])
plt.title('Testing',fontproperties="SimSun",fontsize=24)
# plt.savefig('D:\績效圖/'+ST+'_預測未來'+str(int(n_out/6))+'小時(Testing)(not_process)(TCN).png')
plt.show()
plt.close()

In [ ]:
# RMSE公式：np.sqrt(np.mean((預測值-實際值)^2))
# MAPE公式：100*(np.mean(abs((預測值-實際值)/實際值)))

rmse = []
mape = []

for i in range(transfor_act1.shape[0]-1):
    for j in range(transfor_act1.shape[1]):
        rmse.append((transfor_pred1[i][j]-transfor_act1[i][j])**2)
        if transfor_act1[i][j] == 0:
            mape.append(abs(transfor_pred1[i][j]-transfor_act1[i][j]))
        else:
            mape.append(abs((transfor_pred1[i][j]-transfor_act1[i][j])/transfor_act1[i][j]))
            
rmse = np.sqrt(np.mean(rmse))
mape = np.mean(mape)*100

print('RMSE:'+str(rmse))
print('MAPE:'+str(mape))

In [ ]:
a = []
b = []
c = []
for i in range(image[1].shape[0]):
    for j in range(0,image[1].shape[1],tw):
        a.append(X_test_time[i][j])
        b.append(image[1][i][j])
        c.append(image[3][i][j])

act = np.array(b)
pred = np.array(c)
transfor_act = np.reshape(act,(int(len(act)/transfor_to_hour),transfor_to_hour))
transfor_pred = np.reshape(pred,(int(len(pred)/transfor_to_hour),transfor_to_hour))

In [ ]:
def make_plot(train_yy,test_yy,pred_train_data,pred_test_data):
    number = np.random.randint(test_yy.shape[-1])
    plt.figure(figsize=(5,5))
    plt.plot(train_yy.T[number])
    plt.plot(pred_train_data.T[number])
    plt.title('Training',fontproperties="SimSun",fontsize=24)
    plt.savefig('D:\績效圖/'+ST+'_預測未來'+str(int(n_out/6))+'小時(Training)(maxmin_process)(Conv1D).png')
    plt.show()
    plt.close()
    plt.figure(figsize=(5,5))
    plt.plot(test_yy.T[number])
    plt.plot(pred_test_data.T[number])
    plt.title('Testing',fontproperties="SimSun",fontsize=24)
    plt.savefig('D:\績效圖/'+ST+'_預測未來'+str(int(n_out/6))+'小時(Testing)(maxmin_process)(Conv1D).png')
    plt.show()
    plt.close()

In [ ]:
make_plot(image[0],transfor_act,image[2],transfor_pred)

In [ ]:
# RMSE公式：np.sqrt(np.mean((預測值-實際值)^2))
# MAPE公式：100*(np.mean(abs((預測值-實際值)/實際值)))

rmse = []
mape = []

for i in range(transfor_act.shape[0]):
    for j in range(transfor_act.shape[1]):
        rmse.append((transfor_pred[i][j]-transfor_act[i][j])**2)
        if transfor_act[i][j] == 0:
            mape.append(abs(transfor_pred[i][j]-transfor_act[i][j]))
        else:
            mape.append(abs((transfor_pred[i][j]-transfor_act[i][j])/transfor_act[i][j]))
            
rmse = np.sqrt(np.mean(rmse))
mape = np.mean(mape)*100

print('RMSE:'+str(rmse))
print('MAPE:'+str(mape))

# model_3：TCN

In [ ]:
train_XX = train_X
test_XX = test_X
train_yy = train_y
test_yy = test_y

In [ ]:
model_3 = Sequential([TCN(input_shape=(train_XX.shape[1], train_XX.shape[2]),nb_filters=256, return_sequences=True, dilations=[1, 2, 4, 8, 16, 32]),
        Dense(1)])#use_batch_norm=False,use_layer_norm=False,use_weight_norm=False, dropout_rate=0.3,
model_3.add(Flatten())
model_3.add(Dense(train_yy.shape[-2]))
model_3.compile(optimizer="adam", loss="mse",metrics=tf.keras.metrics.MeanAbsoluteError())
model_3.summary()
history = model_3.fit(train_XX, train_yy, epochs=n_epoch, batch_size=n_batch, verbose=1,shuffle=False)

In [ ]:
image = model_inverse(train_XX,train_yy,test_XX,test_yy,model_3,not_process=True) # 改模型參數 ,prepare_data[2]

In [ ]:
# 10 min
number = np.random.randint(image[1].shape[-1])
plt.figure(figsize=(5,5))
plt.plot(image[1].T[number])
plt.plot(image[3].T[number])
plt.title('Testing',fontproperties="SimSun",fontsize=24)
# plt.savefig('D:\績效圖/'+ST+'_預測未來'+str(int(n_out/6))+'小時(Testing)(not_process)(TCN).png')
plt.show()
plt.close()

In [ ]:
pm10_10_act = image[1]
pm10_10_pred = image[3]

rmse = []
mape = []

for i in range(pm10_10_act.shape[0]):
    for j in range(pm10_10_act.shape[1]):
        rmse.append((pm10_10_pred[i][j]-pm10_10_act[i][j])**2)
        if pm10_10_act[i][j] == 0:
            mape.append(abs(pm10_10_pred[i][j]-pm10_10_act[i][j]))
        else:
            mape.append(abs((pm10_10_pred[i][j]-pm10_10_act[i][j])/pm10_10_act[i][j]))
            
rmse = np.sqrt(np.mean(rmse))
mape = np.mean(mape)*100

print('RMSE:'+str(rmse))
print('MAPE:'+str(mape))

In [ ]:
data1 = pd.read_csv('D:/'+ST+'(已清洗).csv')
time = pd.date_range(start='1/1/2020',end='1/1/2021',freq='H')[:-1]
data1['Time'] = time

num = []
for i in range(len(X_test_time[0])):
    for j in range(len(data1)):
        if data1.loc[j,'Time'] == X_test_time[0][i]:
            num.append(j)
            break
num = num[0]

data2 = data1[num:]
data2.reset_index(inplace=True)
data2 = data2.drop(['index'],axis=1)
data2_time = list(data2['Time'])

a1 = []
b1 = []
c1 = []
for i in range(image[1].shape[0]):
    for j in range(0,image[1].shape[1],tw):
        if j == 0:
            a1.append(X_test_time[i][j])
            b1.append(image[1][i][j])
            c1.append(image[3][i][j])
        else:
            a1.append(X_test_time[i][j])
            b1.append(np.mean(image[1][i][j-tw:j]))
            c1.append(np.mean(image[3][i][j-tw:j]))

act1 = np.array(b1)
pred1 = np.array(c1)
transfor_act1 = np.reshape(act1,(int(len(act1)/transfor_to_hour),transfor_to_hour))
transfor_pred1 = np.reshape(pred1,(int(len(pred1)/transfor_to_hour),transfor_to_hour))

In [ ]:
# 平均1小時
number = np.random.randint(transfor_act1.shape[-1])
plt.figure(figsize=(5,5))
plt.plot(transfor_act1.T[number])
plt.plot(transfor_pred1.T[number])
plt.title('Testing',fontproperties="SimSun",fontsize=24)
# plt.savefig('D:\績效圖/'+ST+'_預測未來'+str(int(n_out/6))+'小時(Testing)(not_process)(TCN).png')
plt.show()
plt.close()

In [ ]:
# RMSE公式：np.sqrt(np.mean((預測值-實際值)^2))
# MAPE公式：100*(np.mean(abs((預測值-實際值)/實際值)))

rmse = []
mape = []

for i in range(transfor_act1.shape[0]-1):
    for j in range(transfor_act1.shape[1]):
        rmse.append((transfor_pred1[i][j]-transfor_act1[i][j])**2)
        if transfor_act1[i][j] == 0:
            mape.append(abs(transfor_pred1[i][j]-transfor_act1[i][j]))
        else:
            mape.append(abs((transfor_pred1[i][j]-transfor_act1[i][j])/transfor_act1[i][j]))
            
rmse = np.sqrt(np.mean(rmse))
mape = np.mean(mape)*100

print('RMSE:'+str(rmse))
print('MAPE:'+str(mape))

In [ ]:
a = []
b = []
c = []
for i in range(image[1].shape[0]):
    for j in range(0,image[1].shape[1],tw):
        a.append(X_test_time[i][j])
        b.append(image[1][i][j])
        c.append(image[3][i][j])

act = np.array(b)
pred = np.array(c)
transfor_act = np.reshape(act,(int(len(act)/transfor_to_hour),transfor_to_hour))
transfor_pred = np.reshape(pred,(int(len(pred)/transfor_to_hour),transfor_to_hour))

In [ ]:
def make_plot(train_yy,test_yy,pred_train_data,pred_test_data):
    number = np.random.randint(test_yy.shape[-1])
    plt.figure(figsize=(5,5))
    plt.plot(train_yy.T[number])
    plt.plot(pred_train_data.T[number])
    plt.title('Training',fontproperties="SimSun",fontsize=24)
    plt.savefig('D:\績效圖/'+ST+'_預測未來'+str(int(n_out/6))+'小時(Training)(not_process)(TCN_4).png')
    plt.show()
    plt.close()
    plt.figure(figsize=(5,5))
    plt.plot(test_yy.T[number])
    plt.plot(pred_test_data.T[number])
    plt.title('Testing',fontproperties="SimSun",fontsize=24)
    plt.savefig('D:\績效圖/'+ST+'_預測未來'+str(int(n_out/6))+'小時(Testing)(not_process)(TCN_4).png')
    plt.show()
    plt.close()

In [ ]:
make_plot(image[0],transfor_act,image[2],transfor_pred)

In [ ]:
# RMSE公式：np.sqrt(np.mean((預測值-實際值)^2))
# MAPE公式：100*(np.mean(abs((預測值-實際值)/實際值)))

rmse = []
mape = []

for i in range(transfor_act.shape[0]):
    for j in range(transfor_act.shape[1]):
        rmse.append((transfor_pred[i][j]-transfor_act[i][j])**2)
        if transfor_act[i][j] == 0:
            mape.append(abs(transfor_pred[i][j]-transfor_act[i][j]))
        else:
            mape.append(abs((transfor_pred[i][j]-transfor_act[i][j])/transfor_act[i][j]))
            
rmse = np.sqrt(np.mean(rmse))
mape = np.mean(mape)*100

print('RMSE:'+str(rmse))
print('MAPE:'+str(mape))